## Assignment 1: Implementation of Stochastic Optimization Methods (SGD + Variance Reduction)##

This jupyter notebook is Assignment 1 for the course "EN.553.662: Optimization for Data Science".

In this assignment we will focus on the practical implementation of Stochastic Gradient Methods for solving popular optimization problems. All algorithms and their convergence guarantees presented in the class. 

You have to complete 2 main tasks each of them being equal to 50 points of the total 100 points of this assignment. 

The Assignemnt is equivalent to 15% of the final grade. 

PLEASE SUBMIT THE ipynb file to CANVAS. Penalty for not doing so (submitting a pdf instead): 10 points.

PLEASE INCLUDE COMMENTS IN YOUR CODE explaining your steps. 

The missing parts of the code which require your input (update in terms of code) are denoted with ............

## 1. Loss functions, gradients and step-sizes

Throughout this assignment we will focus on two classes of of finite-sum problems of the form: 
$$
 \min_{x\in R^d} \left[ f(x) = \frac{1}{n} \sum_{i=1}^n f_i( \langle A_i , x \rangle, b_i) \right],
$$
where $A_i$ is a vector (or matrix) of features and $b_i$ is an observation/label. This as we mention in the lectures corresponds to dataset $$ D= \{ (A_i, b_i ), i = 1,2,\dots , n\},$$ which we assume that has been cleaned (all pairs ($A_i, b_i )$ have the same size and shape). 

The problems that we are interested in are 

##### Ridge regression: 
$$ \min_x f(x)= \frac{1}{n} \sum_{i=1}^n \frac{1}{2} \left( A_i^\top x - b_i \right)^2 + \frac{\lambda}{2} \|x\|^2$$

#####  $\ell_2$ regularized logistic regression:
$$ \min_x f(x)= \frac{1}{n} \sum_{i=1}^n \log \left(1+e^{-b_i \langle A_i , x \rangle} \right) +\frac{\lambda}{2} \|x\|^2$$


Note that both of the above problems can be writen as an optimization problem of the form: 
$$
\frac 1n \sum_{i=1}^n f_i(x)
$$

where for ridge regression: 
$$
f_i(x) = \frac{1}{2} \left( A_i^\top x - b_i \right)^2 + \frac{\lambda}{2} \|x\|^2,
$$
and for logistic regression: 
$$
f_i(x)=\log \left(1+e^{-b_i \langle A_i , x \rangle} \right) +\frac{\lambda}{2} \|x\|^2.
$$

## 2. Generate Data 

Let us first explain what would be the datasets for the problems under study. 
That is explain how the matrix A and vector b (labels) of the main problems are selected. 

In particular we will run the methods for solving both synthetic (Gaussian noise) and real data from LIBSVM datasets (https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/). The synthetic data will be constructed in each individial task. For the real data we will use the following package: 

pip install libsvmdata

from libsvmdata import fetch_libsvm

X,y = fetch_libsvm(“madelon’)

The names of the dataset used in the assignments of our course are "abalone", "w3a", "a1a", "madelon", "gisette", "a9a". To use them in each experiment you will simply need to call the above function and save the matrix X and vector of labels y. 

##  3. Tasks

Having presented the optimization problems of interest and the generation of data. Let us move on to the main tasks of this assignment. First let us import the following standard packages. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
from libsvmdata import fetch_libsvm

### Task 1: Evaluation of SGD (on synthetic and real data) 

Let us start by stating the theorem as presented in the class: 

Assume $f$ is $\mu$-quasi-strongly convex and that $(f,D)\sim ES(\cal L)$.
Choose   $\gamma_k=\gamma \in (0,  \frac{1}{2\cal L}]$ for all $k$. Then iterates of SGD satisfy: 
\begin{equation}\label{eq:convsgd}
\mathbb{E} \| x^k - x^* \|^2 \leq \left( 1 - \gamma \mu \right)^k \| x^0 - x^* \|^2 + \frac{2 \gamma \sigma^2}{\mu}. 
\end{equation}

Having this theorem in mind let us now implement some variants of SGD. 

For all plots in this task, to evaluate SGD use the relative error measure  $\frac{\|x^k - x^* \|^2}{\|x^0 - x^* \|^2}$ and plot its value as the algorithm progress. For all implementations, select the starting point $x^0$ to be the vector of all zeros. Please run each method until a pre-specified maximum number of epochs that you will select in order for the theoretical result to be illustrated. In this task since we only evaluate single-element sampling SGD for the horizontal axis you can use either the number of epochs or the number of iterations. 

For both problems, ridge regression and logistic regression, $\mathbf{A} \in \mathbb{R}^{n \times d}, b \in \mathbb{R}^n$ are the given data and $\lambda > 0$ is the regularization parameter. 
We generated synthetic data in both problems by sampling the rows of matrix $\mathbf{A}$ ($\mathbf{A}[i, :]$)  from the standard Gaussian distribution $\mathcal{N}(0, 1)$. Furthermore 
for ridge regression we sampled the entries of $b$ from the standard Gaussian distribution while in the case of logistic regression $b \in \{-1, 1\}^n$ where $\mathbb{P}(b_i=1) = \mathbb{P}(b_i=-1) = \frac{1}{2}$.
The regularization parameter is selected to be equal to $\lambda=1/n$. 

For synthetic datasets please use $n=1000, d=400$ for the ridge regression problem and $n=2000, d=100$ for the logistic regression problem. For the experiments on real data we will use LIBSVM datasets. In particular select the "abalone" dataset for the ridge regression problem and the "a1a" for the logistic regression problem. 

$\textbf{1.1.}$ (30 points) Design constant step-size SGD with uniform-single elements sampling for solving the least square regression and $\ell_2$-logistic regression problem. 
- (6 points) What are the values of the step-size $\gamma=\frac{1}{2\cal L}$ in this scenario for the two problems? How the value of  $\cal L$ is related to valus $L$ and $L_{max}$ for the case of single-element uniform sampling?
- (12 points) Plot how the trajectory of SGD looks like for the theoretical step-size $\gamma=\frac{1}{2\cal L}$ for the two problems of ridge regression and logistic regresion in the synthetic regime (1 plot for each problem). How the plot is connected to the theoretical result? 
- (12 points) Provide a plot for each problem (synthetic data sets) where you compare SGD with step-sizes $\gamma_1=10^{-3}$, $\gamma_2=10^{-5}$, $\gamma_3=10^{-7}$. Please explain your outputs. How the plot is related to the theoretical result? 

$\textbf{1.2.}$ (10 points) Provide a comparison (generate plots) of "Constant Vs decreasing/switching step-size rule" (choices that were proposed in class) for SGD for solving ridge regression and $\ell_2$-logistic regression problem. What is the main output? Does the methods behave as expected? Here we need to generate two plots for each problem (one plot on synthetic data and one on the real data we explain above).

Recall that the switching step-size rule for strongly convex and smooth problems are: 

\begin{equation}\label{eq:gammakdef}
\gamma_k= 
\begin{cases}
\displaystyle \tfrac{1}{2 \cal L} & \mbox{for}\quad k \leq 4\lceil\mathcal{K} \rceil \\[0.3cm]
\displaystyle \tfrac{2k+1}{(k+1)^2 \mu} &  \mbox{for}\quad k > 4\lceil\mathcal{K} \rceil,
\end{cases}
\end{equation}
where $\mathcal{K} = \cal L /\mu $ .

$\textbf{1.3.}$  (10 points) Provide a Comparison (generate plots) of "Uniform sampling Vs Importance Sampling (single-element regime)" for SGD for solving the two problems under study. What is the output? Please explain your findings. For this task focus only on synthetic datasets (that is generate 1 plot for each problem). 

In [2]:
### Include your solution to TASK 1 at this point (before Task 2)

# You can either define one sinle function of SGD for all TASKS 1.1-1.3 (with several inputs) or you can create 
# different funtions (one for each subtask). Any approach is acceptable. The most important thing is to include
# comments for the graders to be able to go easily through your code. 

# To get an idea of how a more general function would look like we provide an example below 
# of a function of SGD with all possible inputs.


# SGD with single-element sampling
def SGD(x_0, x_min, steps, data, grad, L, Lsum, mu, Lmax, n_iter = 500, trials = 10, epoch=True, decreasing=False, importance=False):
             
    return relative_error_vec

# In the above function with x_min we denote x^* which is needed for the computation of the relative error vector. 
# this can be appoximated in advanced for each problem using determinstic gradient descent 
# for a specific number of iterations. (this should be done before each task)

# The trials = 10 shows have many runs of the method your functions runs. At the end the relative_error_vec 
# will be the average of 10 different runs (as we explained in the class)


In [ ]:
#once the SGD function/or functions is ready you can use a variant like the below 
#for the comparison and to generate the plots.

#here we include the variant for the task 1.1.3

SGD_err1 = SGD(x_init_sys_r, x_min_sys_r, 10**(-3), (X_sys_r, y_sys_r), grad_r, L_i_sys_r, Lsum_sys_r, mu_sys_r, Lmax_sys_r, n_iter = 10000, epoch=False, decreasing=False, importance=False)
SGD_err2 = SGD(x_init_sys_r, x_min_sys_r, 10**(-5), (X_sys_r, y_sys_r), grad_r, L_i_sys_r, Lsum_sys_r, mu_sys_r, Lmax_sys_r, n_iter = 10000, epoch=False, decreasing=False, importance=False)
SGD_err3 = SGD(x_init_sys_r, x_min_sys_r, 10**(-7), (X_sys_r, y_sys_r), grad_r, L_i_sys_r, Lsum_sys_r, mu_sys_r, Lmax_sys_r, n_iter = 10000, epoch=False, decreasing=False, importance=False)

# Plot the Relative Error
plt.plot(np.arange(len(SGD_err1)), SGD_err1, label = 'SGD with 10^(-3)')
plt.plot(np.arange(len(SGD_err2)), SGD_err2, label = 'SGD with 10^(-5)')
plt.plot(np.arange(len(SGD_err3)), SGD_err3, label = 'SGD with 10^(-7)')


plt.yscale('log')
plt.grid(True)
plt.ylabel("Relative Error")
plt.xlabel("Iteration")
plt.title('Trajectory of SGD with constant stepsize for synthetic data of ridge regression')
plt.legend()

### Task 2: On variance-reduced methods: Fast convergence to the exact solution ### 

In the class we present and analyze two variance reduced methods for solving problems similar to ridge regression and logistic regression. These are the L-SVRG and SAGA. In this task we will evaluate the behaviour of these methods.

For all plots in this task, to evaluate SGD and the variance reduced methods, we use the relative error measure  $\frac{\|x^k - x^* \|^2}{\|x^0 - x^* \|^2}$ and plot its value as the algorithms progress. For all implementations, select the starting point $x^0$ to be the vector of all zeros. Please run each method until a pre-specified maximum number of epochs that you will select in order for the theoretical result to be illustrated. In this task since variance reduced methods require a full gradient evaluation in some of the iteration for the horizontal axis you should use the number of epochs (number of stochastic gradient evaluations) for a fair comparison to the vanilla SGD.

$\textbf{3.1.}$ (20 points) As a first sub-task, please design L-SVRG and SAGA method as presented in the class. 

$\textbf{3.2.}$ (30 points) Using the synthetic datasets described in Task 1, compare in a plot, the L-SVRG, the SAGA and the constant step-size SGD (all under uniform single-element sampling) for solving ridge regression and logistic regression problems. For each method use the parameters suggested by the Theorems presented in the class. For this subtask you will need to produce 2 plots (one for each problem). 
Explain your findings. Do the methods behave as the theorems predict?


In [95]:
### Include your solution to TASK 3 at this point (before Task 4)

# L-SVRG

def L_SVRG(....):
 
    return relative_error_vec #average after 10 trials

# SAGA

def SAGA(...):
    
    return relative_error_vec #average after 10 trials

In [ ]:
#Generate the plots

SGD_err = SGD(....)
LSVRG_err = L_SVRG(...)
SAGA_err = SAGA(...) 

# Plot the Relative Error
plt.plot(np.arange(len(SGD_err)), SGD_err, label = 'SGD')
plt.plot(np.arange(len(LSVRG_err)), LSVRG_err, label = 'LSVRG')
plt.plot(np.arange(len(SAGA_err)), SAGA_err, label = 'SAGA')


plt.yscale('log')
plt.grid(True)
plt.ylabel("Relative Error")
plt.xlabel("Epoch")
plt.title('Trajectory of ridge regression')
plt.legend()